In [1]:
%matplotlib nbagg
#%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib import gridspec 
from astropy.modeling import models
from astropy.modeling.models import BlackBody
from astropy import units as u
from astropy.visualization import quantity_support
from numba import jit

import time
dataall=np.load('dataall_save_k3.npy')

box_size_xmin = 1e-01
box_size_xmax = 1e+04
box_size_ymin = 1e-06
box_size_ymin = 1e-20
box_size_ymax = 1e+02
#pis=np.pi*2*1.04e-15*1e+4
pis=np.pi*2
#wav = np.logspace(4,7,27) * u.AA
#wav = np.logspace(3,8,27) * u.AA
#wavf = np.logspace(3,8,27)
wav = np.logspace(3,7,27) * u.AA
wavf = np.logspace(3,7,27)
c=3e8#light speed [m]
#Bkei=((c*1e6)/(wavf*1e6)**2)##old
Bkei=(c/(wav**2))*1e-9#W/m
Bkei=Bkei.value#W/m
print((wav))
cmap=plt.get_cmap("tab10")
obs=np.array([1,2,3,5,6,10,12,16,17])
lifetime=np.ones((30, 10,8))

cmap=plt.get_cmap("tab10")
limK=5
limKhigh=150
limKmid=50
limKlow=1





[1.00000000e+03 1.42510267e+03 2.03091762e+03 2.89426612e+03
 4.12462638e+03 5.87801607e+03 8.37677640e+03 1.19377664e+04
 1.70125428e+04 2.42446202e+04 3.45510729e+04 4.92388263e+04
 7.01703829e+04 1.00000000e+05 1.42510267e+05 2.03091762e+05
 2.89426612e+05 4.12462638e+05 5.87801607e+05 8.37677640e+05
 1.19377664e+06 1.70125428e+06 2.42446202e+06 3.45510729e+06
 4.92388263e+06 7.01703829e+06 1.00000000e+07] Angstrom


In [2]:
tstart = time.time()
pltdat=dataall[:,:,1]
x =pltdat[:,0]
datas=np.zeros((8,10,30,27))
#datas=np.zeros((8,10,300,27))
#timings=np.arange(30)*10#each1Myr 
timings=np.arange(300)#each1Myr 
def BB(T,wavs):
    BB =(BlackBody(temperature=T))(wavs)
    return BB


for inicon in range(8):
    ini=(inicon+1)*10
    print(ini)    
    for i in timings:
        for j in range(10): #10=0.1~1.0 1=1mass
            ininum=ini-9+j
            pltdat=dataall[:,:,ininum]
            s =pltdat[:,i+1]               
            s[s<1e-6]=0 #kagen ika ha 0 ni suru
            t =pltdat[:,i+302]
            tfix=pltdat[:,400]#BB_fix用
            m=0
            l=0
            bb = BlackBody(temperature=10*u.K)
            fluxall = bb(wav)*0 #initial
            #for l in range(125):#10au以内
            while ( l<126):
                l=m##     
                flux=BB(t[l]*u.K,wav)
                #kei=(1-np.exp(-(10*s[l])))*x[l]*(x[l+1]-x[l])##   
                kei=(1-np.exp(-s[l]))*x[l]*(x[l+1]-x[l])##    
                F=flux*kei*pis*Bkei##
                Fu=F.unit
                #print(F.unit)
                fluxall+=F##     
                m+=1##
            fluxs=np.zeros((27,4000))
            while (l>125 and l<3998):
                l=m##
                if i ==0:#ini only
                    #print(fluxs[:,l])
                    fluxs[:,l]=BB(t[l]*u.K,wav)
                    #print(fluxs[:,l])
                '''flux=BB(t[l]*u.K,wav)
                print(flux.shape)'''
                #kei=(1-np.exp(-(10*s[l])))*x[l]*(x[l+1]-x[l])##    
                kei=(1-np.exp(-s[l]))*x[l]*(x[l+1]-x[l])##    
                #F=flux*kei*pis*Bkei##
                F=fluxs[:,l]*kei*pis*Bkei*Fu##
                fluxall+=F##
                m+=1##
            #print(num)
            for waves in range(27):
                datas[inicon,j,int(i/10),waves]=fluxall[waves].value #30
                #datas[inicon,j,i,waves]=fluxall[waves].value #300
print(datas)
np.save('SED_k20.npy', datas)

10


/home/porco/anaconda3/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in expm1
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/porco/anaconda3/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


20
30
40
50
60
70
80
[[[[4.56982356e-035 5.78304033e-029 7.50386990e-025 ... 1.85555250e-023
    4.90859681e-024 1.26404887e-024]
   [2.03992236e-035 3.22415546e-029 4.86054734e-025 ... 1.81822204e-023
    4.81794338e-024 1.24205268e-024]
   [1.26748147e-035 2.28512524e-029 3.76403562e-025 ... 1.80415382e-023
    4.78379129e-024 1.23376735e-024]
   ...
   [3.21526099e-037 1.62308209e-030 5.28913520e-026 ... 1.77054466e-023
    4.70219934e-024 1.21397044e-024]
   [3.00909688e-037 1.54783168e-030 5.10592011e-026 ... 1.77040793e-023
    4.70185331e-024 1.21388389e-024]
   [2.82305679e-037 1.47867130e-030 4.93547367e-026 ... 1.77025728e-023
    4.70146931e-024 1.21378736e-024]]

  [[3.08120484e-031 2.72104484e-026 5.38170203e-023 ... 2.79107780e-023
    7.19884557e-024 1.82198116e-024]
   [2.15901451e-031 2.09286756e-026 4.39593501e-023 ... 2.70705856e-023
    6.99488874e-024 1.77250206e-024]
   [1.79755919e-031 1.82849768e-026 3.96102193e-023 ... 2.67701113e-023
    6.92196533e-024 1.7548